In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
import rhinoMorph
from konlpy.tag import Okt
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [2]:
#크롤된 뉴스 데이터 불러오기
columns = ['media','date','title','article_original','url']
news_crawled_df = pd.read_excel('./crawled_news/news_df_210222_v05.xlsx')
news_crawled_df

,media,date,title,article_original,url
0,조선일보,2020-01-02,150만명 몰린 타임스스퀘어 새해 무대도 BTS,2012년 싸이에 이어 두 번째 ABC방송 라이브 쇼에서 생중계 전 지구를 홀린 ...,https://news.naver.com/main/read.nhn?mode=LSD&...
1,조선일보,2020-01-02,방탄소년단 CNN 선정 2010년대 음악 변화시킨 아티스트,방탄소년단BTS이 미국 CNN 선정 2010년대 음악을 변화시킨 10대 아티스트에...,https://news.naver.com/main/read.nhn?mode=LSD&...
2,조선일보,2020-01-03,방탄소년단 새앨범 작업중 새해도 K팝 인베이전 이어진다,트와이스 도쿄돔 공연 블랙핑크도 새앨범 예정 경자년 새해에도 K팝 스타들의 세계...,https://news.naver.com/main/read.nhn?mode=LSD&...
3,조선일보,2020-01-03,서울시 BTS트와이스 활용해 K팝 관광명소 추천,K팝에 대한 관심이 전 세계적으로 높아지면서 서울시가 이른바 K팝 명소들을 선정해...,https://news.naver.com/main/read.nhn?mode=LSD&...
4,조선일보,2020-01-06,봉준호 BTS 영향력은 나의 3천배 멋진 아티스트의 나라,골든글로브 시상식에서 한국 최초로 외국어영화상을 수상한 기생충의 봉준호 감독이 한...,https://news.naver.com/main/read.nhn?mode=LSD&...
...,...,...,...,...,...
856,한겨레,2020-12-26,방탄학BTSology BTS 연구는 이미 시작되었다,방탄소년단BTS의 전세계적인 인기는 이제 낯설지 않다. 2017년 처음 미국 무대...,https://news.naver.com/main/read.nhn?mode=LSD&...
857,한겨레,2020-12-26,세계의 모든 힘없는 자들에게 2020 BTS 혁명,이 세계의 모든 힘없는 자들아 우리가 패배할 날이 올지도 모르지만 오늘은 아니야 ...,https://news.naver.com/main/read.nhn?mode=LSD&...
858,조선일보,2020-12-29,코로나로 힘든 1년 트롯맨봉준호BTS 있어 웃고 울었다,2020년 잊을 수 없는 문화계의 10일 2020년은 한국 문화계의 저력과 바닥이...,https://news.naver.com/main/read.nhn?mode=LSD&...
859,중앙일보,2020-12-30,박영선 삶은 아무 일 없단 듯 계속된다BTS 노래로 신년사,박영선 중소벤처기업부 장관은 30일 발표한 신년사에서 이미 최선을 다 하고 계신데...,https://news.naver.com/main/read.nhn?mode=LSD&...


In [3]:
news_crawled_df.isna().sum()

media               0
date                0
title               0
article_original    8
url                 0
dtype: int64

In [4]:
news_crawled_df.loc[news_crawled_df.article_original.isna()]

,media,date,title,article_original,url
157,중앙일보,2020-03-22,민기자야 BTS 기사 쓴 기자가 직접 자기 기사 댓글 읽었다,NaN,https://news.naver.com/main/read.nhn?mode=LSD&...
211,조선일보,2020-05-15,사진 BTS RM이 남긴 미술관 방명록쉽지 않은 시기 같이 이겨내요,NaN,https://news.naver.com/main/read.nhn?mode=LSD&...
405,한겨레,2020-09-02,영상 BTS 빌보드 핫 100 1위온라인 글로벌 기자간담회,NaN,https://news.naver.com/main/read.nhn?mode=LSD&...
407,경향신문,2020-09-02,BTS Makes It over the Final Billboard Threshold,NaN,https://news.naver.com/main/read.nhn?mode=LSD&...
665,한겨레,2020-10-14,BTS 소속사의 주식 상장이 논란에 싸인 이유 간단정리 공모주 내손안의Q 19,NaN,https://news.naver.com/main/read.nhn?mode=LSD&...
668,경향신문,2020-10-14,Will BTS Be Allowed to Postpone Enrollment Mil...,NaN,https://news.naver.com/main/read.nhn?mode=LSD&...
728,중앙일보,2020-11-02,Hot Poll 방탄소년단 병역특례에 대한 중앙일보 온라인 핫 폴 결과,NaN,https://news.naver.com/main/read.nhn?mode=LSD&...
764,경향신문,2020-11-20,불행했지만 행복했던 한해 좌절 이겨낸 힘은 멤버들 BTS 글로벌 기자간담회 일문일답,NaN,https://news.naver.com/main/read.nhn?mode=LSD&...


In [5]:
#drop na
news_crawled_df.dropna(inplace=True)
news_crawled_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 853 entries, 0 to 860
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   media             853 non-null    object
 1   date              853 non-null    object
 2   title             853 non-null    object
 3   article_original  853 non-null    object
 4   url               853 non-null    object
dtypes: object(5)
memory usage: 40.0+ KB


In [6]:
# reset index
news_crawled_df.reset_index(inplace=True)
news_crawled_df.drop('index',inplace=True,axis=1)

In [7]:
# \n과 같은 불필요한 문자 cleaning
# news_crawled_df['article_original'] = news_crawled_df['article_original'].apply(lambda x : re.sub(r'[^A-Za-z0-9가-힣 ]','',x))

In [8]:
news_crawled_df

,media,date,title,article_original,url
0,조선일보,2020-01-02,150만명 몰린 타임스스퀘어 새해 무대도 BTS,2012년 싸이에 이어 두 번째 ABC방송 라이브 쇼에서 생중계 전 지구를 홀린 ...,https://news.naver.com/main/read.nhn?mode=LSD&...
1,조선일보,2020-01-02,방탄소년단 CNN 선정 2010년대 음악 변화시킨 아티스트,방탄소년단BTS이 미국 CNN 선정 2010년대 음악을 변화시킨 10대 아티스트에...,https://news.naver.com/main/read.nhn?mode=LSD&...
2,조선일보,2020-01-03,방탄소년단 새앨범 작업중 새해도 K팝 인베이전 이어진다,트와이스 도쿄돔 공연 블랙핑크도 새앨범 예정 경자년 새해에도 K팝 스타들의 세계...,https://news.naver.com/main/read.nhn?mode=LSD&...
3,조선일보,2020-01-03,서울시 BTS트와이스 활용해 K팝 관광명소 추천,K팝에 대한 관심이 전 세계적으로 높아지면서 서울시가 이른바 K팝 명소들을 선정해...,https://news.naver.com/main/read.nhn?mode=LSD&...
4,조선일보,2020-01-06,봉준호 BTS 영향력은 나의 3천배 멋진 아티스트의 나라,골든글로브 시상식에서 한국 최초로 외국어영화상을 수상한 기생충의 봉준호 감독이 한...,https://news.naver.com/main/read.nhn?mode=LSD&...
...,...,...,...,...,...
848,한겨레,2020-12-26,방탄학BTSology BTS 연구는 이미 시작되었다,방탄소년단BTS의 전세계적인 인기는 이제 낯설지 않다. 2017년 처음 미국 무대...,https://news.naver.com/main/read.nhn?mode=LSD&...
849,한겨레,2020-12-26,세계의 모든 힘없는 자들에게 2020 BTS 혁명,이 세계의 모든 힘없는 자들아 우리가 패배할 날이 올지도 모르지만 오늘은 아니야 ...,https://news.naver.com/main/read.nhn?mode=LSD&...
850,조선일보,2020-12-29,코로나로 힘든 1년 트롯맨봉준호BTS 있어 웃고 울었다,2020년 잊을 수 없는 문화계의 10일 2020년은 한국 문화계의 저력과 바닥이...,https://news.naver.com/main/read.nhn?mode=LSD&...
851,중앙일보,2020-12-30,박영선 삶은 아무 일 없단 듯 계속된다BTS 노래로 신년사,박영선 중소벤처기업부 장관은 30일 발표한 신년사에서 이미 최선을 다 하고 계신데...,https://news.naver.com/main/read.nhn?mode=LSD&...


In [ ]:
#불용어 import
stopwords2 = pd.read_csv('./불용어리스트2.txt',header=None)
stopwords = list(stopwords2[0])
stopwords

In [10]:
#토크나이징
def okt_morph(articles):
    okt = Okt()
    articles_morphed = []
    for article in articles:
        morph = okt.nouns(article)
        
        stopped = ''
        for word in morph:
            if word not in stopwords:
                stopped += word
                stopped += ' '
                
        articles_morphed.append(stopped)
    return articles_morphed

In [11]:
#형태소 분해된 컬럼 추가
news_crawled_df['article_morphed'] = okt_morph(news_crawled_df.article_original)
#날짜 칼럼 추가
news_crawled_df['date_pd']=pd.to_datetime(news_crawled_df['date'])

In [14]:
news_crawled_df.article_morphed[0]

'싸이 두 방송 라이브 쇼 생중계 전 지구 홀린 그룹 사회자 소개 방탄소년단 모습 마지막 날 미국 뉴욕 맨해튼 타임스스퀘어 무대 선 저녁 타임스스퀘어 볼드롭 대형 크리스털 볼 신년 카운트다운 행 보기 위해 사람 곳 오전 매년 날 최소 곳 올해 때문 열기 더욱 뉴욕 경찰 날 인파 추정 한국 가수 타임스스퀘어 새해 맞이 무대 오른 건 싸이 두 번 밤 열정 춤 노래 시선 전 인파 싸이 강남스타일 말춤 날 타임스스퀘어 전 세계 아미 팬 위 시 야광 봉 한국어 떼창 선사 공연 뒤 리더 살 홀로 집 영화 광경 눈앞 감격 멤버 목소리 해피 뉴 공연 미국 최대 새해 맞이 라이브 쇼 방송 뉴 어스 로킹 이브 통 생중계 최정 상급 가수 출연 유명 프로그램 올해 외 컨트리 가수 샘 헌트 싱어 송 라이터 앨러 니스 모리 세트 무대 사전 녹화 통해 할리우드 무대 출연 타임스스퀘어 무대 직접 선 처음 행사 공동 진행 라이언 시크레스트 올해 타임스스퀘어 절반 팬 언급 인기 입증 듯 날 오후 타임스스퀘어 주변 곳곳 노래 행인 노래 몸 새벽 줄 에블린 잭슨 재능 사려 이루 말 수 뉴욕 해 마지막 날 타임스스퀘어 콘서트장 보이 보도 '

In [12]:
#원본 date column drop
# news_crawled_df.drop(columns='date',axis=1,inplace=True)

In [19]:
#일 년 기간을 12구간으로 cut하여 기간에 따른 기사들 그룹화
date_grouped = []
date_range = pd.date_range(start='2020-01-01', end='2020-12-31',periods=13)
L = len(date_range)
R = len(news_crawled_df)

for i, date in enumerate(date_range):
    index_group = []
    if i< L-1:
        for j in range(R):
            if date_range[i] < news_crawled_df.date_pd[j] <= date_range[i+1]:
                index_group.append(j)
        date_grouped.append(index_group)

In [20]:
date_range

DatetimeIndex(['2020-01-01 00:00:00', '2020-01-31 10:00:00',
               '2020-03-01 20:00:00', '2020-04-01 06:00:00',
               '2020-05-01 16:00:00', '2020-06-01 02:00:00',
               '2020-07-01 12:00:00', '2020-07-31 22:00:00',
               '2020-08-31 08:00:00', '2020-09-30 18:00:00',
               '2020-10-31 04:00:00', '2020-11-30 14:00:00',
               '2020-12-31 00:00:00'],
              dtype='datetime64[ns]', freq=None)

In [21]:
# 그룹별 tfidf 변환
def tfidfy(time_period_no):
    tfidf = TfidfVectorizer(lowercase=False)
    #그룹에 있는 아티클 가져오기
    article_group = []
    for index in date_grouped[time_period_no]:
      article_group.append(news_crawled_df.article_original[index])
    
    art_group_tfidf = tfidf.fit_transform(article_group)
    return art_group_tfidf

In [22]:
#date 그룹 내에서 유사도 높은 기사의 index 가져오는 함수
def get_similar_art_index(group_no, art_group_tfidf, group_index):
    # 그룹 안의 한 기사와 나머지 기사간의 similarity 비교
    sim_pair = cosine_similarity(art_group_tfidf[group_index],art_group_tfidf)
    # 인덱스 내림차순으로 소트
    sorted_index = sim_pair.argsort()[:,::-1]
    # 자기자신은 빼기
    sorted_index = sorted_index[:,1:]
    #print(sorted_index)
    
    #유사도가 큰 순으로 유사도 값을 재정렬하되 자기 자신은 제외
    art_sim_value = np.sort(sim_pair.reshape(-1))[::-1]
    art_sim_value = art_sim_value[1:]
    #print(art_sim_value)
    
    #기사의 real 인덱스의 reference point 구하기
    ref = 0
    for i in range(group_no):
        ref += len(date_grouped[i])

    # 유사도가 0.3보다 높은 real 인덱스만 반환하고 해당 유사 카운터 증가.
    index_high = []
    for idx, value in enumerate(art_sim_value):
         if value>=0.3:
            group_index = sorted_index[0][idx]
            real_index = ref + group_index
            index_high.append(real_index)
            #유사 article로 참조된 횟수 카운터
            sim_counter[real_index] += 1
            
    return index_high

In [23]:
#유사 참조 카운터
sim_counter = np.zeros(len(news_crawled_df))
#전체 문서에 대해, 날짜 구간 당 유사한 기사의 index를 리스트에 저장하고 카운터를 늘려준다.
sim_list=[]
for group_no in range(len(date_grouped)):
  for idx in range(len(date_grouped[group_no])):
    art_group_tfidf = tfidfy(group_no)
    sim_list.append(get_similar_art_index(group_no, art_group_tfidf, idx))

#dataframe에 컬럼 추가
news_crawled_df['sim_articles'] = sim_list
news_crawled_df['sim_count'] = sim_counter

In [18]:
#유사한 기사가 3개 이상이면 불러오기  --> 없어도 되는 단계
idx_list = []
for i,ls in enumerate(news_crawled_df['sim_articles']):
  if len(ls)>=3:
    idx_list.append(i)
#print(idx_list)
news_crawled_df.iloc[idx_list]

,media,title,article_original,url,article_morphed,date_pd,sim_articles,sim_count
36,조선일보,방탄소년단 그래미 무대 선다 한국 가수 최초,그룹 방탄소년단 BTS 이 제62회 그래미 어워즈 Grammy Awards 에서 공...,https://news.naver.com/main/read.nhn?mode=LSD&...,그룹 방탄소년단 BTS 62회 그래미 어워즈 Grammy Awards 공연 한 다 ...,2020-01-24,"[37, 41, 39, 40]",4.0
37,경향신문,방탄소년단 한국 가수 최초 미 그래미 어워드 공연 확정,그룹 방탄소년단 BTS 이 제62회 그래미 어워드 Grammy Awards 에서 공...,https://news.naver.com/main/read.nhn?mode=LSD&...,그룹 방탄소년단 BTS 62회 그래미 어워드 Grammy Awards 공연 한 다 ...,2020-01-24,"[36, 41, 39, 40]",4.0
39,한겨레,BTS 한국 가수 최초 그래미 공연내년엔 후보 목표,그룹 방탄소년단이 한국 가수 최초로 미국 그래미 시상식에서 공연했다 방탄소년단은 ...,https://news.naver.com/main/read.nhn?mode=LSD&...,그룹 방탄소년단 한국 가수 최초 미국 그래미 시상식 공연 방탄소년단 26일 현지 미...,2020-01-27,"[40, 41, 43, 36, 37]",5.0
40,경향신문,내년엔 그래미 후보될까짧았던 방탄소년단 무대,방탄소년단이 26일 현지시간 열린 그래미 시상식 공연 무대에 올랐다 단독 공연이 아...,https://news.naver.com/main/read.nhn?mode=LSD&...,방탄소년단 26일 현지 열린 그래미 시상식 공연 무대 올랐다 단독 공연 아닌 합동 ...,2020-01-27,"[39, 43, 41, 37, 36]",5.0
41,조선일보,방탄소년단 한국 가수 최초 그래미 공연 내년엔 후보 도전,그룹 방탄소년단 BTS 이 26일 현지 시각 한국 가수 최초로 미국 그래미 시싱식에...,https://news.naver.com/main/read.nhn?mode=LSD&...,그룹 방탄소년단 BTS 26일 현지 한국 가수 최초 미국 그래미 시 싱 식 공연 방...,2020-01-27,"[39, 36, 37, 40, 43, 44]",6.0
...,...,...,...,...,...,...,...,...
811,중앙일보,한국말 노래로 핫 100 정상 대관식BTS 빌보드 62년 역사 다시 썼다,방탄소년단 BTS 이 또 한 번 새 역사를 썼다 우리말로 된 신곡 라이프 고스 온 ...,https://news.naver.com/main/read.nhn?mode=LSD&...,방탄소년단 BTS 한 번 새 역사 썼다 우리말 된 신곡 라이프 고스 온 Life G...,2020-12-02,"[800, 807, 802, 812, 808]",5.0
822,중앙일보,BTS MAMA 8관왕 싹쓸이저희만 좋은 소식 마음 무겁기도,그룹 방탄소년단 BTS 이 엠넷 아시안 뮤직 어워즈 2020 MAMA 에서 2년 연...,https://news.naver.com/main/read.nhn?mode=LSD&...,그룹 방탄소년단 BTS 엠넷 아시안 뮤직 어워즈 2020 MAMA 2년 연속 대상 ...,2020-12-07,"[825, 824, 823]",3.0
823,조선일보,BTS MAMA 싹쓸이,그룹 방탄소년단 BTS 이 아시아 최고 권위의 케이팝 시상식 2020 MAMA 엠넷...,https://news.naver.com/main/read.nhn?mode=LSD&...,그룹 방탄소년단 BTS 아시아 최고 권위 케이팝 시상식 2020 MAMA 엠넷 아시...,2020-12-07,"[822, 825, 824]",3.0
824,한겨레,BTS 올해도 MAMA서 대상 싹쓸이8개 부문 석권,그룹 방탄소년단 BTS 이 엠넷 아시안 뮤직 어워즈 2020 MAMA 에서 2년 연...,https://news.naver.com/main/read.nhn?mode=LSD&...,그룹 방탄소년단 BTS 엠넷 아시안 뮤직 어워즈 2020 MAMA 2년 연속 대상 ...,2020-12-07,"[825, 822, 823]",3.0


In [24]:
#대표 기사 뽑기 (가장 참조가 많이 된 기사)
articles_final = []
for group in date_grouped:
    indx = group[0]
    for art_indx in group:
        if news_crawled_df.sim_count[art_indx] >= news_crawled_df.sim_count[indx]:
            indx = art_indx
    articles_final.append(indx)
articles_final

[41, 121, 149, 170, 214, 264, 305, 353, 544, 630, 773, 792]

In [25]:
# 뽑은 대표 기사 갯수
len(articles_final)

12

In [26]:
# from gensim.summarization import summarize
from lexrankr import LexRank
# 출력
# 1. 유사 카운트
# 2. 날짜
# 3. 기사 제목
# 4. 기사 본문
# 5. 요약본
for indx in articles_final:
    sim_count = news_crawled_df.sim_count[indx]
    if sim_count>=2:
#         lexrank = LexRank()
#         lexrank.summarize(news_crawled_df.article_original[indx])
#         summaries = lexrank.probe(3)
        print('sim_count: ', sim_count)
        print('article index: ', indx)
        print(news_crawled_df.date_pd[indx])
        print(news_crawled_df.title[indx],'\n')
        print(news_crawled_df.article_original[indx],'\n')
#         print(summaries,'\n')

sim_count:  5.0
article index:  41
2020-01-27 00:00:00
방탄소년단 한국 가수 최초 그래미 공연 내년엔 후보 도전 

 그룹 방탄소년단BTS이 26일현지 시각 한국 가수 최초로 미국 그래미 시싱식에서 공연했다. 방탄소년단은 이날 미국 로스앤젤레스LA 스테이플스 센터에서 열린 제62회 그래미 시상식 2부에서 래퍼 릴 나스 엑스 컨트리 가수 빌레 레이 사이러스와 함께 올드 타운 로드 올스타즈Old Town Road AllStars 무대를 선보였다. 릴 나스 엑스가 먼저 자신의 히트곡 올드 타운 로드를 부르기 시작했다. 회전식 무대가 움직이며 서울을 연상케하는 배경이 펼쳐지자 방탄소년단이 등장해 서울 버전으로 각색한 서울 타운 로드 노래를 이어 부르기 시작했다. 이 노래는 방탄소년단 멤버 RM이 지난해 7월 올드 타운 로드에 참여해 만든 음원이다. 무대 배경에는 서울 도심의 빌딩과 BTS 방탄소년단의 팬클럽 아미 등의 글자가 찍힌 모습이 보여 미국 내 방탄소년단의 인기를 실감케했다. 무대 마지막에는 빌리 레이 사이러스 메이슨 램지 디플로 등이 방탄소년단과 함께 올드 타운 로드 후렴구를 부르며 무대를 달궜다. 이번 공연으로 방탄소년단은 2년 연속 그래미 시상식에 참석하게 됐다. 방탄소년단은 지난해 최우수 RB 앨범 부문 시상자로 그래미 시상식 무대에 올랐다. 당시 방탄소년단은 이곳에 다시 서겠다며 그래미상 도전 의지를 보였으나 올해 후보에는 오르지 못했다. 방탄소년단은 이날 레드카펫 행사에서 내년 그래미 시상식 후보에 오르는 것이 목표라고 포부를 밝혔다. 그래미상은 세계 최고 권위의 대중음악 시상식이다. 방탄소년단은 그래미와 함께 미국 3대 대중음악 시상식으로 통하는 빌보드 뮤직 어워즈와 아메리칸 뮤직 어워즈에서 수상한 바 있다. 올해 그래미 시상식에서는 영국의 10대 여성 가수 빌리 아일리시가 올해의 레코드 앨범 노래 부문과 신인상까지 4개 부문을 수상하며 주인공으로 우뚝 섰다. 빌리 아일리시는 지난해 히트곡 배드 가이로 세

In [27]:
#합치기 위해 정리
articles_final_df = news_crawled_df.iloc[articles_final]
# articles_final_df.drop(columns=['article_morphed','sim_articles','sim_count'],axis=1,inplace=True)
articles_final_df= articles_final_df[['media','date','title','article_original','url']]
articles_final_df

,media,date,title,article_original,url
41,조선일보,2020-01-27,방탄소년단 한국 가수 최초 그래미 공연 내년엔 후보 도전,그룹 방탄소년단BTS이 26일현지 시각 한국 가수 최초로 미국 그래미 시싱식에서 ...,https://news.naver.com/main/read.nhn?mode=LSD&...
121,경향신문,2020-02-28,BTS 월드투어 4월 서울 콘서트 취소 코로나19 확산 비상,일부 팬들 환불받아 성금 기탁 트와이스 등 공연 취소 잇따라 코로나19의 확산으...,https://news.naver.com/main/read.nhn?mode=LSD&...
149,조선일보,2020-03-16,방탄소년단 빌보드 200 8위3주 연속 10위권,그룹 방탄소년단BTS이 미국 빌보드 메인 앨범 차트인 빌보드 200에서 8위를 차...,https://news.naver.com/main/read.nhn?mode=LSD&...
170,경향신문,2020-04-09,방탄소년단 음반 누적판매 2032만장한국 가요 최다 기록,그룹 방탄소년단사진이 앨범 누적 판매량 2000만장을 넘기면서 한국 가요 역사상 ...,https://news.naver.com/main/read.nhn?mode=LSD&...
214,조선일보,2020-05-18,이태원 아이돌 BTS 정국 아스트로 차은우 NCT 재현 세븐틴 민규였다,최근 소문으로만 떠돌던 이태원발 코로나 집단 감염 기간 이태원 방문 아이돌이 그룹...,https://news.naver.com/main/read.nhn?mode=LSD&...
264,경향신문,2020-06-08,BTS 유튜브 주최 가상 졸업식 디어 클래스 오브 2020 축사 꽃다발은 없어도 가...,그룹 방탄소년단BTS이 코로나19 확산으로 오프라인 졸업식에 가지 못하는 졸업생들...,https://news.naver.com/main/read.nhn?mode=LSD&...
305,경향신문,2020-07-23,BTS 75만명 본 방방콘 더 라이브로 기네스 세계기록 추가,방탄소년단의 온라인 콘서트 방방콘 더 라이브이하 방방콘가 세계에서 가장 많은 시청...,https://news.naver.com/main/read.nhn?mode=LSD&...
353,경향신문,2020-08-30,방탄소년단 신곡 영국 오피셜 차트 3위,그룹 방탄소년단BTS사진의 신곡 다이너마이트Dynamite가 세계 양대 팝 차트 ...,https://news.naver.com/main/read.nhn?mode=LSD&...
544,경향신문,2020-09-29,3주 만에 다시 빌보드 정상에 선 방탄소년단,그룹 방탄소년단BTS사진의 다이너마이트가 3주 만에 미국 빌보드 메인 싱글 차트인...,https://news.naver.com/main/read.nhn?mode=LSD&...
630,경향신문,2020-10-13,병무청 대중문화우수자 병역 연기 입법 추진확정시 BTS 연기 가능,병무청이 13일 대중문화예술 분야 우수자의 병역 징집소집 연기 등을 담은 병역법 ...,https://news.naver.com/main/read.nhn?mode=LSD&...


In [28]:
# csv로 save
articles_final_df.to_csv('articles_final_peter.csv',mode='w',index=False)